## 載入套件

In [13]:
# predict
import os,shutil,sys#建資料夾用
import time #sleep用
import argparse
import json
import cv2
from utils.utils import get_yolo_boxes, makedirs
from utils.bbox import draw_boxes
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math
import pandas as pd
from keras.models import load_model#載入h5檔案使用
import tensorflow as tf
import glob

## 初始設定載入

In [11]:
'''
首先需要一個training後的.h5結構權重檔案
參考權重檔(另一種載入方法)
https://stackoverflow.com/questions/53295570/userwarning-no-training-configuration-found-in-save-file-the-model-was-not-c
'''
infer_model = load_model('H:\\predict_model\\imt_yolo3.h5',compile=False)

In [14]:
#建立資料夾
def emtydir(dirname):
    if os.path.isdir(dirname):
        shutil.rmtree(dirname)
        time.sleep(2)
    os.mkdir(dirname)
#測試資料夾
def load_dataset(path):
    try:
        file=glob.glob(path+'*.jpg')
        count=0
        for i,f in enumerate(file):
            count=count+i
        print("路徑正常有",i,"個影像")
    except:
        print("檔案問題")

### Anchors設置(不需要)

In [ ]:
ROOT_PATH = 'H:/predict_model/'
# 數量
CLUSTERS = 6
# 影像尺寸
SIZE = 640

# 格式
def load_dataset(path):
    jpegimages = os.path.join(path, 'JPEGImages')
    if not os.path.exists(jpegimages):
        print('沒照片')
        sys.exit(0)

    label_file = os.listdir(labels_txt)
    print('label count: {}'.format(len(label_file)))
    dataset = []

    for label in label_file:
        with open(os.path.join(labels_txt, label), 'r') as f:
            txt_content = f.readlines()

        for line in txt_content:
            line_split = line.split(' ')
            roi_with = float(line_split[len(line_split)-2])
            roi_height = float(line_split[len(line_split)-1])
            if roi_with == 0 or roi_height == 0:
                continue
            dataset.append([roi_with, roi_height])
            # print([roi_with, roi_height])

    return np.array(dataset)

data = load_dataset(ROOT_PATH)
out = kmeans(data, k=CLUSTERS)

print(out)
print("Accuracy: {:.2f}%".format(avg_iou(data, out) * 100))
print("Boxes:\n {}-{}".format(out[:, 0] * SIZE, out[:, 1] * SIZE))

ratios = np.around(out[:, 0] / out[:, 1], decimals=2).tolist()
print("Ratios:\n {}".format(sorted(ratios)))

## 比對缺值

In [5]:
#比對兩list是否有不重複值
def different(list_1, list_2): 
    return (list(set(list_1).symmetric_difference(set(list_2))))

## 製作撈座標function

In [6]:
#傳boxes進來，判斷label以及過濾重複方框
def coordinate(boxes,threshold):
    #print("==========驗證==========")
    labels = ['Mark_X','Mark_Y','X','Y'] 
    #box字典
    label_list=[]
    #比對
    compare=[]
    #存放結果表格，用字典建
    label_dir={'label':[],
             'x1':[],
             'y1':[],
             'x2':[],
             'y2':[],
             'score':[],
            }
    label_table=pd.DataFrame(label_dir)
    for m in range(len(boxes)):
        if boxes[m].score !=-1:
            label=boxes[m].label
            if label==0:
                label_name="Mark_X"
            elif label==1:
                label_name='Mark_Y'
            elif label==2:
                label_name="X"
            elif label==3:
                label_name="Y"
            if(boxes[m].score>threshold):
                #新增label進list
                label_list.append(label_name)
                #print(label_name,boxes[m].xmin,boxes[m].ymin,boxes[m].xmax,boxes[m].ymax,"Score",boxes[m].score)
            #將數值存入統一字典格式    
            statistics_dir={'label':[label_name],
                              'x1':[boxes[m].xmin],
                              'y1':[boxes[m].ymin],
                              'x2':[boxes[m].xmax],
                              'y2':[boxes[m].ymax],
                              'score':[boxes[m].score],
                                 }
            #將每一筆box資料新增到global 的結果字典
            statistics_tabel=pd.DataFrame(statistics_dir)
            label_table=label_table.append(statistics_tabel,ignore_index=True)
            

    #找list中有沒有包含所有label
    compare=different(label_list,labels)
    #print("==========過濾結果===========")
    if len(compare)<1:
        #過濾重複數值(先照信心值排序，照label中重複的只留下第一筆，最後按照label排序
        label_table=label_table.sort_values('score',ascending=False).drop_duplicates('label',keep='first').sort_values(by='label')
        #重新編排index
        label_table.reset_index(drop=True,inplace=True)
        #print(label_table)
        #開始量測(座標表格,標記名稱)，回傳量測結果與屬於哪個角落
        point_table,corner=cutting_measure(label_table,labels)
        return point_table,corner
    else:
        #print("有缺值",compare)
        #有缺label停止程式
        return 0,0

### 量測精度

In [7]:
#量測經度
def cutting_measure(table,ref_list):
        #initialize
        point_1=0
        point_2=0
        point_3=0
        point_4=0
        point_5=0
        point_6=0
        point_7=0
        point_8=0
        #放8個點位置
        point_dic={'sheet_id':[],
                   'point_1':[],
                   'point_2':[],
                   'point_3':[],
                   'point_4':[],
                   'point_5':[],
                   'point_6':[],
                   'point_7':[],
                   'point_8':[],
                   'check':[],
                  }
        #將字典做成表格
        point_table=pd.DataFrame(point_dic)

        print("===========================")
        #過濾完成開始計算距離，取過濾完的dataframe就一定剩下4個
        c_row,c_column=table.shape
        #順序(Mark_X、Mark_Y、X、Y)


        data_x1=table['x1']
        data_x2=table['x2']
        data_y1=table['y1']
        data_y2=table['y2']
        
        
        #算Mark_X
        Mark_X_Line=data_y1[0]+((data_y2[0]-data_y1[0])/2)
        #算Mark_Y
        Mark_Y_Line=data_x1[1]+((data_x2[1]-data_x1[1])/2)
        #算X_Line
        X_Line=data_y1[2]+((data_y2[2]-data_y1[2])/2)
        #算Y_Line
        Y_Line=data_x1[3]+((data_x2[3]-data_x1[3])/2)
        
        #量測距離，用切割線-mark 
        x_measure=X_Line-Mark_X_Line
        y_measure=Y_Line-Mark_Y_Line
        
        #利用正負判斷角落
        #-x ,-y，進不去
        
        if x_measure<0 and y_measure<0:
            corner="Upper_left_corner"
            point_2=abs(x_measure)
            point_3=abs(y_measure)
            step_dic={ 
                       'point_2':[point_2],
                       'point_3':[point_3],
                     }
        #-x ,+y
        if x_measure<0 and y_measure>0:
            corner="Upper_right_corner"
            point_1=abs(x_measure)
            point_8=abs(y_measure)
            step_dic={ 
                       'point_1':[point_1],
                       'point_8':[point_8],
                     }

        #+x ,+y
        if y_measure>0 and x_measure>0:
            corner="Bottom_right_corner"
            point_6=abs(x_measure)
            point_7=abs(y_measure)
            step_dic={ 
                       'point_6':[point_6],
                       'point_7':[point_7],
                     }
        #+x ,-y，進不去
        if y_measure<0 and x_measure>0 :
            corner="Bottom_left_corner"
            point_5=abs(x_measure)
            point_4=abs(y_measure)
            step_dic={ 
                       'point_5':[point_5],
                       'point_4':[point_4],
                     }
        #輸出哪個角落
        print(corner)
        #把結果放到字典

        step_table=pd.DataFrame(step_dic)
        #合併dataframe(新增時忽略index欄位，並不要排序)
        point_table=point_table.append(step_table,ignore_index=True,sort=False)
        #轉換行跟列，方便計算
        transpose= point_table.T
        #把null的值都補上0
        transpose_point_table=transpose.where(transpose.notnull(),0)
        #存成csv(省略index)
        #df=pd.DataFrame(transpose_point_table).to_csv('H:/Record/point.csv',index=False)
        #回傳表跟屬於哪個角落
        return transpose_point_table,corner

## 換算真實座標

In [4]:
#換算距離成真實數值
def real_measure(table):
    #放8個點位置
    point_dic={'sheet_id':[],
                'point_1':[],
                'point_2':[],
                'point_3':[],
                'point_4':[],
                'point_5':[],
                'point_6':[],
                'point_7':[],
                'point_8':[],
                'check':[],
              }
    point_table=pd.DataFrame(point_dic)
    #轉置
    transpose=point_table.T
    #取size
    c_row,c_column=table.shape
    #取欄位
    point_1=table.iloc[1,0]
    point_2=table.iloc[2,0]
    point_3=table.iloc[3,0]
    point_4=table.iloc[4,0]
    point_5=table.iloc[5,0]
    point_6=table.iloc[6,0]
    point_7=table.iloc[7,0]
    point_8=table.iloc[8,0]

    #開始換算
    stendard=450
    point_1=stendard+(point_1*4.34487)
    point_2=stendard+(point_2*4.41068)
    point_3=stendard+(point_3*1.90193)
    point_4=stendard+(point_4*2.12035)
    point_5=stendard+(point_5*4.54456)
    ##3.26950
    point_6=stendard+(point_6*5.13992)
    point_7=stendard+(point_7*2.38112)
    ##3.75535
    point_8=stendard+(point_8*2.28308)
    
    real_point_dic={
                    "point_1":[point_1],
                    "point_2":[point_2],
                    "point_3":[point_3],
                    "point_4":[point_4],
                    "point_5":[point_5],
                    "point_6":[point_6],
                    "point_7":[point_7],
                    "point_8":[point_8],
                    }
    real_point_table=pd.DataFrame(real_point_dic)
    transform=real_point_table.T
    
    
    print("=========real===========")
    print(transform)
    return(transform)

## model繪製方框

In [3]:
def predict_x_ray(input_path,output_path):
    # Set some parameter
    net_h, net_w = 608, 608 # a multiple of 32, the smaller the faster
    #物件信心值、重疊度
    obj_thresh,nms_thresh=0.60,0.3

    anchors = [55,69, 75,234, 133,240, 136,129, 142,363, 203,290, 228,184, 285,359, 341,260]
    #[0],[1],[2]照字母順序編排標籤順序
    labels = ['Mark_X','Mark_Y','X','Y'] 
    #####################################################
    #存每個角落的2個量測點
    total_dic={'sheet_id':[0],
                           'point_1':[0],
                           'point_2':[0],
                           'point_3':[0],
                           'point_4':[0],
                           'point_5':[0],
                           'point_6':[0],
                           'point_7':[0],
                           'point_8':[0],
                           'check':[0],
                }
    total_table=pd.DataFrame(total_dic)
    #轉換行跟列，方便計算
    transpose= total_table.T
    #空值補0
    transpose_total_table=transpose.where(transpose.notnull(),0)
    #角落參數
    corner_list=['Upper_left_corner','Upper_right_corner','Bottom_right_corner','Bottom_left_corner']
    #放角落比對
    corner_input_list=[]
    #比對
    compare=[]
    #########################################################

    # Read file
    image_paths = []
    if os.path.isdir(input_path): 
        for inp_file in os.listdir(input_path):
            image_paths += [input_path + inp_file]
    else:
        image_paths += [input_path]

    image_paths = [inp_file for inp_file in image_paths if (inp_file[-4:] in ['.jpg', '.png', '.JPG'])]

    # the main loop
    for image_path in image_paths:

        image = cv2.imread(image_path)
        print(image_path)

    # predict the bounding boxes
        boxes = get_yolo_boxes(infer_model, [image], net_h, net_w, anchors, obj_thresh, nms_thresh)[0]
    # draw bounding boxes on the image using labels
        draw_boxes(image, boxes, labels, obj_thresh)  

    # write the image with bounding boxes to file
        output_img_path = output_path +'predict_'+image_path.split('\\')[-1]
        cv2.imwrite(output_img_path, np.uint8(image))
        img=cv2.imread(output_img_path)[:,:,::-1]
        #print(output_img_path)

    #取得BBOX座標
        #box(boxes)
        label_table,corner=coordinate(boxes,obj_thresh)
        #將corner放到比對重複角落list
        corner_input_list.append(corner)
        #若缺label停止執行
        if type(label_table)==int:
            return 0
        else:
            #修改欄位名稱(方便後面相加的時候識別)
            label_table.columns=[1]
            #相加
            print("==========================")

            #連接兩個字典
            df_result=pd.concat([transpose_total_table,label_table],axis=1,ignore_index=False)
            #原本的表格加上新增的表格
            df_result[0]=df_result[0]+df_result[1]
            #刪除新增的欄
            df_result.pop(1)
            #將存好的table存到外面
            transpose_total_table = df_result
            #print(transpose_total_table)
    #找list中有沒有包含所有的角落
    compare=different(corner_input_list,corner_list)
    #都有包含所有角落
    if len(compare)<1:
        print(transpose_total_table)
        #換算真實尺寸
        real_measure(transpose_total_table)
        #存成csv(省略index)
        transpose_total_table=pd.DataFrame(transpose_total_table).to_csv('H:/Record/point.csv',index=False)
    else:
        #print("有缺值",compare)
        return 0

## 使用

In [2]:
#### 使用 ####
#輸入進來的測試路徑
input_path   = 'H:\\test\\'
#標框結果存檔路徑
output_path  = 'H:\\Result\\'
#新建Folder
#emtydir(output_path)
load_dataset('H:\\test\\')
predict_x_ray(input_path,output_path)
'''
路徑正常有 6 個影像
H:\test\t001_03.jpg.jpg
===========================
Bottom_left_corner
==========================
H:\test\t002_03.jpg.jpg
===========================
Bottom_right_corner
==========================
H:\test\t003_03.jpg.jpg
===========================
Upper_left_corner
==========================
H:\test\t004_03.jpg.jpg
===========================
Upper_right_corner
==========================
             0
sheet_id   0.0
point_1   23.0
point_2   24.0
point_3   41.5
point_4   42.0
point_5   18.5
point_6   19.0
point_7   32.0
point_8   32.5
check      0.0
=========real===========
                  0
point_1  549.932010
point_2  555.856320
point_3  528.930095
point_4  539.054700
point_5  534.074360
point_6  547.658480
point_7  526.195840
point_8  524.200100
'''

## 測試輸出座標(單一)

In [1]:
#test
def box(boxes):
    for m in range(len(boxes)):
        if boxes[m].score !=-1:
            label=boxes[m].label
            if label==0:
                label_name="Mark_X"
                Mark_X_Line=boxes[m].ymax-boxes[m].ymin
                print("Mark_X",Mark_X_Line)
            elif label==1:
                label_name="Mark_Y"
                Mark_Y_Line=boxes[m].xmax-boxes[m].xmin
                print("Mark_Y",Mark_Y_Line)
            elif label==2:
                label_name="X"
                X_Line=boxes[m].ymax-boxes[m].ymin
                print("X",X_Line)
            elif label==3:
                label_name="Y"
                Y_Line=boxes[m].xmax-boxes[m].xmin
                print("Y",Y_Line)
            if(boxes[m].score>0.80):
                print(label_name,boxes[m].xmin,boxes[m].ymin,boxes[m].xmax,boxes[m].ymax,"評分",boxes[m].score)
            
#輸出
'''
X    1 -3 188 509 194
Y    2 234 29 251 291
Mark 0 244 158 327 192
X    1 -25 185 329 197
Y    2 237 113 251 432
'''

## dataframe 測試

In [ ]:
## test 
dic1={
    "ccc":[1,0,0,0,0,0,0,1]
}
dic2={
    "bbb":[0,1,0,1,0,0,0,0]
}
df_new=pd.DataFrame(dic1)
df_new2=pd.DataFrame(dic2)
#連接新增兩個字串
df_new3=pd.concat([df_new,df_new2],axis=1,ignore_index=False)
#矩陣相加
df_new3['ccc']=df_new3["ccc"]+df_new3["bbb"]
#刪除欄位
df_new3.pop('bbb')
print(df_new3)
#先列在欄
point_1=df_new3.iloc[0,0]
point_2=df_new3.iloc[1,0]
point_3=df_new3.iloc[2,0]
print(point_1,"==",point_2,"==",point_3)